In [7]:
import torch

In [2]:
print(torch.backends.cudnn.enabled)

True


In [3]:
print(torch.cuda.is_available())

True


In [ ]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [1]:
import os as _os
# import pandas as pd
import pyspark.pandas as ps
audio_file_dir = _os.path.join('data', 'calls', '911_first6sec')
audio_file_csv = _os.path.join(audio_file_dir, '911_metadata_6sec.csv')
df = ps.read_csv(audio_file_csv)

C:\Users\USER\.conda\envs\gpu_torch\Lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "C:\Users\USER\AppData\Roaming\Python\Python311\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\USER\.conda\envs\gpu_torch\Lib\socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exc

Py4JError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext

In [2]:
def get_text_from_audio(audio_file_name):
    import speech_recognition as sr
    import os as _os
    
    r = sr.Recognizer()
    dirr, name = audio_file_name.split('/')
    audio_file_path = _os.path.join('data', 'calls', dirr, name)
    with sr.AudioFile(audio_file_path) as src:
        r.adjust_for_ambient_noise(src, duration=0.5)    
        audio = r.record(src)
        return r.recognize_whisper(audio, model='medium')

In [9]:
get_text_from_audio('911_first6sec/call_2_0.wav')

C:\Users\USER\AppData\Roaming\Python\Python311\site-packages\whisper\timing.py:57: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit


' My mom had a bad time.'

In [10]:
%%timeit
get_text_from_audio('911_first6sec/call_2_0.wav')

9.65 s ± 51.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
text_series = df['filename'][:10].apply(get_text_from_audio)

2min 32s ± 14.4 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
from pandarallel import pandarallel

In [4]:
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [11]:
text_series = df['filename'][:10].parallel_apply(get_text_from_audio)

RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 16777216 bytes.